In [ ]:
!pip install image
!pip install pillow

In [1]:
from pymongo import MongoClient
from gridfs import GridFS
from gridfs import GridFSBucket

import urllib.request
from io import BytesIO
from PIL import Image

#### Text 파일 저장, 불러오기

In [ ]:
#python_test DB가 없으면 새로 만듬
db = MongoClient().python_test

#MongoDB 연동
fs = GridFS(db)

#rb: ReadBinary(한글은 숫자로 변환)
with open('c:/ai/workspace/mongodb/test.txt', 'rb') as f:
    print('f=', f)
    print('f.read()=', f.read())

#MongoDB에 파일f test2로 파일f 내용 저장
with open('c:/ai/workspace/mongodb/test.txt', 'rb') as f:
    fs.put(f, filename='test2.txt')
    
db.fs.files.find()
list(db.fs.files.find())
fs.get_last_version(filename='test.txt')

#### DB Image 저장, 불러오기

In [ ]:
#Image 불러오기
url = 'http://image_url'
data = urllib.request.urlopen(url).read()
Image.open(BytesIO(data))

In [ ]:
#DB Image 저장
db = MongoClient().python_test
bucket = GridFSBucket(db)

#이미지 파일명, 메타데이터 입력
grid_in = bucket.open_upload_stream(
    '43a8a87814b98b5346192ec9855f5883.jpg', 
    metadata={'contentType':'image/jpg'}
)

grid_in.write(data)
grid_in.close()

In [ ]:
db = MongoClient().python_test
fs = GridFS(db)

db.fs.files.find()
db.fs.files.find()[7]
'''print  
{'_id': ObjectId('608f80a4e136bf6b6ee0005a'),
 'filename': '43a8a87814b98b5346192ec9855f5883.jpg',
 'metadata': {'contentType': 'image/jpg'},
 'chunkSize': 261120,
 'md5': 'c5274a1f27f65df228abb2000c267317',
 'length': 155845,
 'uploadDate': datetime.datetime(2021, 5, 3, 4, 48, 38, 715000)}
'''
db.fs.files.find()[7]['filename']

f = fs.get_last_version(filename=db.fs.files.find()[7]['filename'])
data = f.read()
Image.open(BytesIO(data))

#### Naver API

In [ ]:
#네이버 이미지 검색 URL
url = 'https://openapi.naver.com/v1/search/image?query=고양이'

#OpenAPI를 사용하기 위한 ID와 PW 필요
requests.get(url)
'''print
<Response [401]>
'''


#ex)네이버 API 인증 정보
client_id = "udzt3jcCA8xTKi3UjGH_" 
client_secret = "HAk0g9n8cs"

api_id_pw = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

requests.get(url, headers=api_id_pw)
'''print
<Response [200]>
'''
result = requests.get(url, headers=api_id_pw)

In [ ]:
result.json()
result.json()['items']
result.json()['items'][0]['link']

In [ ]:
#keyward:검색어, display:출력수, start:시작점

def get_api_result(keyward, display, start):
    url = 'https://openapi.naver.com/v1/search/image?query={0}&display={1}&start={2}'.format(keyward, display, start)
    result = requests.get(url, headers=api_id_pw)
    return result.json()

In [ ]:

def call_and_print(keyward, total_page):
    link_lst = []
    for page in range(total_page + 1):
        display = 100
        start = (page * 100) + 1
        json_obj = get_api_result(keyward, display, start)
        for item in json_obj['items']:
            link_lst.append(item['link'])
    return link_lst

In [ ]:
call_and_print('고양이', 9)
call_and_print('강아지', 9)